In [ ]:
from slack_sdk import WebClient

from utils.secrets import get_secret
from utils.message_kit import (
    post_regular_message,
    post_ephemeral_message,
    post_block_formatted_message,
    find_channel,
    pull_messages_from_oldest_to_latest,
)

In [ ]:
SLACK_BOT_TOKEN: str = get_secret('bot_user_oauth_token')
OPENAI_TOKEN: str = get_secret('slackbot_openai_svc_acct_secretkey')
MY_SLACK_USER_ID: str = get_secret('yj_slack_user_id')

client = WebClient(token=SLACK_BOT_TOKEN)
SLACK_CHANNEL_ID: str = find_channel(client=client, channel_name='...')

In [ ]:
import time
from typing import TypedDict

Message = TypedDict('Message', {'user': str, 'ts': str, 'thread_ts': str, 'text': str})

def _process_message(message: dict) -> Message:
    if not message.get('thread_ts'):
        return _parse_single_message(message)
    return _parse_message_thread(message)

def _parse_single_message(message: dict) -> Message:
    return {
        'user': message.get('user'),
        'ts': message.get('ts'),
        'thread_ts': message.get('thread_ts'),
        'text': message.get('text'),
    }

def _parse_message_thread(message: dict) -> list[Message]:
    replies = client.conversations_replies(
        channel=SLACK_CHANNEL_ID,
        ts = message.get('thread_ts')
    )
    # return replies.get('messages')
    parse_replies = [_parse_single_message(x) for x in replies.get('messages')]
    return parse_replies

def pull_all_messages_from_oldest_to_latest(
        client: WebClient, 
        channel_id: str, 
        oldest: float, 
        latest: float
) -> list[Message]:
    result = client.conversations_history(
        channel=channel_id, 
        latest=latest, 
        oldest=oldest
    )
    ## Get all top level messages
    root_messages = result.get('messages')
    processed_messages = [_process_message(x) for x in root_messages]
    return processed_messages